In [1]:
import os
import sys

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:\users\17")\
    .getOrCreate()


In [2]:
celsius = [39.2, 36.5, 37.3, 37.8]
def c2f(c) : 
    return (float(9)/5)*c+32

f=map(c2f, celsius)
print f  #map함수를 구현하면 for문이 필요없다.

[102.56, 97.7, 99.14, 100.03999999999999]


In [7]:
sentence = "Hello World"
map(lambda x : x.split(),sentence) #문장을 split하면 글자 단위로 분리

[['H'], ['e'], ['l'], ['l'], ['o'], [], ['W'], ['o'], ['r'], ['l'], ['d']]

In [8]:
sentence_0 =["Hello World", "Good Morning"]
map(lambda x : x.split(), sentence_0) #list를 split하면 단어 단위로 분리

[['Hello', 'World'], ['Good', 'Morning']]

In [9]:
fib=[0,1,1,2,3,5,8,13,21,34,55]
result = filter(lambda x : x%2, fib)
print result

[1, 1, 3, 5, 13, 21, 55]


In [10]:
reduce(lambda x, y : x+y, range(0, 101))

5050

In [13]:
#본격적인 RDD API 사용

In [16]:
nRdd=spark.sparkContext.parallelize([1,2,3,4])
squared = nRdd.map(lambda x : x*x)
print squared

PythonRDD[1] at RDD at PythonRDD.scala:48


In [19]:
#위의 print squared는 rdd를 출력한 것이다.

In [18]:
print squared.collect() #collect action을 통해 값을 출력

[1, 4, 9, 16]


In [20]:
myRdd100 = spark.sparkContext.parallelize(range(1, 101))
myRdd100.reduce(lambda x,y : x+y)

5050

In [21]:
#RDD는 단순 통계 기능을 가지고 있다.

In [23]:
print "sum:", nRdd.sum()
print "min:", nRdd.min()
print "max:", nRdd.max()
print "standard deviation:", nRdd.stdev()
print "variance:", nRdd.variance()

sum: 10
min: 1
max: 4
standard deviation: 1.118033988749895
variance: 1.25


In [28]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data", "ds_spark_wiki.txt"))
myRdd_spark=myRdd2.filter(lambda line: "Spark" in line)
print "How many lines having 'Spark':", myRdd_spark.count()

myRdd_spark.collect()

How many lines having 'Spark': 4


[u'Apache Spark is an open source cluster computing framework.',
 u'Apache Spark Apache Spark Apache Spark Apache Spark',
 u'the Spark codebase was later donated to the Apache Software Foundation,',
 u'Spark provides an interface for programming entire clusters with']

In [29]:
#filter api는 RDD 안에 있는 문장을 조건에 맞게 추출하여 따로 저장이 가능하다.

In [30]:
myRdd_unicode = myRdd2.filter(lambda line: u"스파크" in line)
print myRdd_unicode.first()

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


In [31]:
#stopword(불용어)를 filter를 이용하여 없애보자

In [32]:
stopwords = ['is', 'am', 'are', 'a', 'an', 'the', 'for', 'at']
myRdd_stop=myRdd2.flatMap(lambda x:x.split())\
    .filter(lambda x : x not in stopwords)

In [34]:
for words in myRdd_stop.collect():
    print words,

Wikipedia Apache Spark open source cluster computing framework. 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. Apache Spark Apache Spark Apache Spark Apache Spark 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 Originally developed University of California, Berkeley's AMPLab, Spark codebase was later donated to Apache Software Foundation, which has maintained it since. Spark provides interface programming entire clusters with implicit data parallelism and fault-tolerance.


In [35]:
#print words는 단어를 줄단위로 출력, words,는 띄어쓰기하며 출력

In [36]:
spark.sparkContext.parallelize([1,2,3,4,5,]).foreach(lambda x:x+1)

In [37]:
spark.sparkContext.parallelize([1,2,3,4,5]).map(lambda x:x+1).collect()

[2, 3, 4, 5, 6]

In [38]:
#foreach는 transform인 map함수와 같은 역할을 하는 action 함수이며
#결과값을 출력하지 않는다.

In [39]:
sentences=myRdd2.map(lambda x : x.split(" "))

In [41]:
sentences.count() #count는 문장의 개수를 출력한다.

10

In [42]:
def mySplit(x):
        return x.split(" ")



In [44]:
sentences2=myRdd2.map(mySplit)

In [45]:
sentences2.count()

10

In [46]:
#이렇게 하면 한줄마다 함수를 만들지 않고 사용자 정의 함수로 반복된 함수 처리 가능

In [48]:
sentences.take(3)

[[u'Wikipedia'],
 [u'Apache',
  u'Spark',
  u'is',
  u'an',
  u'open',
  u'source',
  u'cluster',
  u'computing',
  u'framework.'],
 [u'\uc544\ud30c\uce58',
  u'\uc2a4\ud30c\ud06c\ub294',
  u'\uc624\ud508',
  u'\uc18c\uc2a4',
  u'\ud074\ub7ec\uc2a4\ud130',
  u'\ucef4\ud4e8\ud305',
  u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.']]

In [54]:
for line in sentences.collect():
    for words in line:
        print words,
    print "\n---------"

Wikipedia 
---------
Apache Spark is an open source cluster computing framework. 
---------
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. 
---------
Apache Spark Apache Spark Apache Spark Apache Spark 
---------
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 
---------
Originally developed at the University of California, Berkeley's AMPLab, 
---------
the Spark codebase was later donated to the Apache Software Foundation, 
---------
which has maintained it since. 
---------
Spark provides an interface for programming entire clusters with 
---------
implicit data parallelism and fault-tolerance. 
---------


In [55]:
#처음엔 이해가 안됐었는데 이게 for문을 쓰면 한 줄마다 인식을 하나 봄.
#그다음에 문장을 인식해 출력하는데 이게 어떻게 되는지 공부 필요
#map으로 만들어진 rdd 'sentences'는 단어들이 구분된 2차원 리스트라고 생각하면 
#line을 행으로, words를 열로 생각하면 된다.

In [56]:
myRdd2.map(lambda s:len(s)).collect()#map안에 다양한 함수를 넣을 수 있다

[9, 59, 32, 51, 31, 72, 71, 30, 64, 46]

In [73]:
myList=["this is", "a line"]
_rdd=spark.sparkContext.parallelize(myList)

In [59]:
wordsRdd=_rdd.map(lambda x:x.split())
print wordsRdd.collect()

[['this', 'is'], ['a', 'line']]


In [74]:
repRdd=_rdd.map(lambda x: x.replace("a", "AA"))
repRdd.collect() # 문장 안에 있는 a를 AA로 대체

['this is', 'AA line']

In [75]:
upperRDD = wordsRdd.map(lambda x:x[0].upper())
print upperRDD.collect() #subset의 첫번째 단어를 대문자로 만든 뒤 저장

['THIS', 'A']
